In [1]:
# Import
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# from fastai.structured import add_datepart

# Linear Regression
from sklearn.linear_model import LogisticRegression

# Data Collecting

In [2]:
# Config
symbl = 'AAPL'
apiKey = ''

# Fetch Historical Data
df = pd.read_csv('https://www.alphavantage.co/query?datatype=csv&function=TIME_SERIES_MONTHLY&symbol=' + symbl + '&outputsize=compact&apikey=' + apiKey)

# # Use local data
# df = pd.read_csv('data/monthly_MSFT.csv')

# Setting index as date
df['timestamp'] = pd.to_datetime(df.timestamp, format='%Y-%m-%d')
df.index = df['timestamp']

In [5]:
# Write dataframe to access locally
df.to_csv("monthly_AAPL.csv", sep='\t', index=False)

# Data Overview

In [3]:
# Get general statistics
df.describe()

,open,high,low,close,volume
count,240.000000,240.000000,240.000000,240.000000,2.400000e+02
mean,171.693043,184.031872,157.858013,171.610692,2.422656e+09
std,162.912658,171.765951,150.861050,162.092658,1.672617e+09
min,14.200000,14.950000,12.720000,14.140000,3.023817e+08
25%,56.470000,63.868750,47.621250,56.355000,1.132850e+09
50%,119.340000,126.725000,107.985000,120.345500,2.062930e+09
75%,203.815000,216.200000,189.072500,201.657500,3.267391e+09
max,671.160000,705.070000,656.000000,667.105000,1.035668e+10


In [4]:
# Get data types
df.dtypes

timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume                int64
dtype: object

In [6]:
# Check top 10 rows
df.head(10)

,timestamp,open,high,low,close,volume
timestamp,,,,,,
2019-10-14,2019-10-14,225.07,238.1342,215.1300,235.87,302381695
2019-09-30,2019-09-30,206.43,226.4200,204.2200,223.97,542567100
2019-08-30,2019-08-30,213.90,218.0300,192.5800,208.74,681081600
2019-07-31,2019-07-31,203.17,221.3700,198.4100,213.04,473851700
2019-06-28,2019-06-28,175.60,201.5700,170.2700,197.92,515218700
2019-05-31,2019-05-31,209.88,215.3100,174.9900,175.07,739456600
2019-04-30,2019-04-30,191.64,208.4800,188.3800,200.67,506117700
2019-03-29,2019-03-29,174.28,197.6900,169.5000,189.95,650981400
2019-02-28,2019-02-28,166.96,175.8700,165.9333,173.15,472540600


In [7]:
# Check last 10 rows
df.tail(10)

,timestamp,open,high,low,close,volume
timestamp,,,,,,
2000-08-31,2000-08-31,50.313,61.5000,44.250,60.938,1409021600
2000-07-31,2000-07-31,52.125,60.6285,46.875,50.813,1436692600
2000-06-30,2000-06-30,81.750,103.9380,50.313,52.375,2026301200
2000-05-31,2000-05-31,124.875,126.2500,81.750,84.000,2451937600
2000-04-28,2000-04-28,135.500,139.5000,104.875,124.063,2165601200
2000-03-31,2000-03-31,118.563,150.3750,114.000,135.813,2174589200
2000-02-29,2000-02-29,104.000,119.9380,97.000,114.625,1829945600
2000-01-31,2000-01-31,104.875,121.5000,86.500,103.750,3138794400
1999-12-31,1999-12-31,101.000,118.0000,91.063,102.813,2354553600


# Convert closing price to trading action

In [8]:
# Expected Return
# expectedReturn = 0 # Long if the stock price stays the same or increases
expectedReturn = 0.025 # Long if the stock price increases by 2.5%

df['prev_close'] = df['close'].shift(-1)

df['action'] = np.nan 

for i, row in df.iterrows():
    realReturn = (df.loc[i, 'close'] / df.loc[i, 'prev_close']) - 1
    df.loc[i, 'action'] = 1 if (realReturn >= expectedReturn) else 0 # 1 = Long / 0 = Short


df.head(10)

,timestamp,open,high,low,close,volume,prev_close,action
timestamp,,,,,,,,
2019-10-14,2019-10-14,225.07,238.1342,215.1300,235.87,302381695,223.97,1.0
2019-09-30,2019-09-30,206.43,226.4200,204.2200,223.97,542567100,208.74,1.0
2019-08-30,2019-08-30,213.90,218.0300,192.5800,208.74,681081600,213.04,0.0
2019-07-31,2019-07-31,203.17,221.3700,198.4100,213.04,473851700,197.92,1.0
2019-06-28,2019-06-28,175.60,201.5700,170.2700,197.92,515218700,175.07,1.0
2019-05-31,2019-05-31,209.88,215.3100,174.9900,175.07,739456600,200.67,0.0
2019-04-30,2019-04-30,191.64,208.4800,188.3800,200.67,506117700,189.95,1.0
2019-03-29,2019-03-29,174.28,197.6900,169.5000,189.95,650981400,173.15,1.0
2019-02-28,2019-02-28,166.96,175.8700,165.9333,173.15,472540600,166.44,1.0


# Prepare data to merge with sentimental analysis

In [9]:
cropped_df = df[(df['timestamp'].dt.year >= 2000)]
cropped_df.tail(10)

,timestamp,open,high,low,close,volume,prev_close,action
timestamp,,,,,,,,
2000-10-31,2000-10-31,26.688,26.7500,17.500,19.563,5476447200,25.750,0.0
2000-09-29,2000-09-29,61.313,64.1250,25.375,25.750,3629232600,60.938,0.0
2000-08-31,2000-08-31,50.313,61.5000,44.250,60.938,1409021600,50.813,1.0
2000-07-31,2000-07-31,52.125,60.6285,46.875,50.813,1436692600,52.375,0.0
2000-06-30,2000-06-30,81.750,103.9380,50.313,52.375,2026301200,84.000,0.0
2000-05-31,2000-05-31,124.875,126.2500,81.750,84.000,2451937600,124.063,0.0
2000-04-28,2000-04-28,135.500,139.5000,104.875,124.063,2165601200,135.813,0.0
2000-03-31,2000-03-31,118.563,150.3750,114.000,135.813,2174589200,114.625,1.0
2000-02-29,2000-02-29,104.000,119.9380,97.000,114.625,1829945600,103.750,1.0


In [10]:
cropped_df = cropped_df.reset_index(drop=True)
# Drop last row
cropped_df = cropped_df[:-1]

In [11]:
cropped_df.tail(5)

,timestamp,open,high,low,close,volume,prev_close,action
232,2000-06-30,81.750,103.938,50.313,52.375,2026301200,84.000,0.0
233,2000-05-31,124.875,126.250,81.750,84.000,2451937600,124.063,0.0
234,2000-04-28,135.500,139.500,104.875,124.063,2165601200,135.813,0.0
235,2000-03-31,118.563,150.375,114.000,135.813,2174589200,114.625,1.0
236,2000-02-29,104.000,119.938,97.000,114.625,1829945600,103.750,1.0
